# Test forward_projector.py
## Test the principle of forward_projector: to find intersected voxels with point-focused beam and the computation of fwd_peaks and making projections
## Haixing Fang
## Jan 2025

In [ ]:
import os

os.environ['OMP_NUM_THREADS'] = '1'
os.environ['OPENBLAS_NUM_THREADS'] = '1'
os.environ['MKL_NUM_THREADS'] = '1'

exec(open('/data/id11/nanoscope/install_ImageD11_from_git.py').read())
PYTHONPATH = setup_ImageD11_from_git( ) # ( os.path.join( os.environ['HOME'],'Code'), 'ImageD11_git' )

In [ ]:
# import functions we need

import shutil
import concurrent.futures

# %matplotlib ipympl

import h5py
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

In [ ]:
import ImageD11.parameters
import ImageD11.unitcell
import time
from joblib import Parallel, delayed

In [ ]:
from ImageD11.forward_model import forward_projector
from ImageD11.forward_model import io
from ImageD11.forward_model import forward_model
from ImageD11.forward_model import pars_conversion
from ImageD11.forward_model import grainmaps

In [ ]:
import logging
import tqdm

In [ ]:
beam = forward_projector.beam(energy = 43.56, FWHM = [1e-3, 1e-3])

In [ ]:
beam.set_beam_shape(plot_flag=True)

In [ ]:
# sample_filename = '../../A2050_DN_W340_nscope/A2050_DN_W340_nscope_full_slice/A2050_DN_W340_nscope_full_slice_grains.h5'
# sample_filename = 'pbp_tensormap_refined.h5'
sample_filename = '/data/visitor/ma6288/id11/20241119/PROCESSED_DATA/A2050_DN_W340_nscope_5pct_strained/A2050_DN_W340_nscope_5pct_strained_full_slice/DS.h5'
pars_filename = '/data/visitor/ma6288/id11/20241119/PROCESSED_DATA/nscope_pars/pars.json'

In [ ]:
output_folder = 'output_test'
# Check if the folder exists, then delete it
if os.path.exists(output_folder) and os.path.isdir(output_folder):
    shutil.rmtree(output_folder)
    print(f"Deleted folder: {output_folder}")
os.mkdir(output_folder)
print(f"{output_folder} has been created now.")

In [ ]:
sample = forward_projector.sample(filename=sample_filename)
sample.set_rou()
sample.set_mass_abs()

In [ ]:
plt.figure()
plt.imshow(sample.DS['labels'][0,:,:])

In [ ]:
pars = ImageD11.parameters.read_par_file(pars_filename)

In [ ]:
ucell = ImageD11.unitcell.unitcell([4.04761405231186, 4.04761405231186, 4.04761405231186, 90.0, 90.0, 90.0], 225)

# Check intersected voxels

In [ ]:
dty = 100.0
y0_offset = 0.0
voxel_size = sample.DS['voxel_size']
ray_size = np.mean(beam.FWHM) * 1000
omega = 45.0

mask = (sample.DS['labels'] > -1) & (~np.isnan(sample.DS['U'][:, :, :, 0, 0]))
mask = np.moveaxis(mask, 0, 2)

In [ ]:
intersected_sampos, intersected_labpos, intersected_voxels = forward_projector.intersected_sample_pos(mask, dty=dty, y0_offset=y0_offset, voxel_size=voxel_size,
                                                                 omega=omega, ray_size=ray_size, weight = beam.weight, weight_pos = beam.weight_pos,
                                                                 plot_flag=True,detector='eiger')
print(intersected_voxels.shape)

In [ ]:
plt.figure()
plt.hist(intersected_voxels[:,3])

In [ ]:
f, a = plt.subplots(1, 2, figsize=(12, 6))
a[0].scatter(intersected_sampos[:,0], intersected_sampos[:,1], s = 3)
a[0].set_title("intersected_sampos")

a[1].scatter(intersected_labpos[:,0], intersected_labpos[:,1], s = 3)
a[1].set_title("intersected_labpos")
plt.show()

In [ ]:
# compute intersected voxels at different omega angles
dty = 120
omegas = [0, 30, 60, 90, 120, 150]
intersected_voxels_all = []
for omega in omegas:
    intersected_sampos, intersected_labpos, intersected_voxels = forward_projector.intersected_sample_pos(mask, dty=dty, y0_offset=y0_offset, voxel_size=voxel_size,
                                                                 omega=omega, ray_size=ray_size, weight = [1.0, 0.8, 0.5, 0.13], weight_pos = [0.5, 1.0, 1.5, 2.3], plot_flag=False,detector='eiger')
    intersected_voxels_all.append(intersected_voxels)

In [ ]:
# plot intersected voxels at different omega angles
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection="3d")

grid_size = mask.shape
voxel_indices = np.argwhere(mask > 0)
indices = np.array(voxel_indices)
ax.scatter(indices[:, 0], indices[:, 1], indices[:, 2], c="blue", s=1, alpha=0.03, label="Masked Voxels")

colors = plt.cm.tab20(np.linspace(0, 1, len(intersected_voxels_all)))  # Use a colormap for variety

for i, (intersected, omega) in enumerate(zip(intersected_voxels_all, omegas)):
    ax.scatter(
        intersected[:, 0],
        intersected[:, 1],
        intersected[:, 2],
        c=colors[i],  # Assign a unique color to each intersected group
        s=8,
        alpha=0.9,
        label="Intersected voxels @" + str(omega) + " deg",
    )
    
# ax.plot(ray_path[:, 0], ray_path[:, 1], ray_path[:, 2], "r-", label="Ray Path", linewidth=2)
ax.set_xlim(0, grid_size[0])
ax.set_ylim(0, grid_size[1])
ax.set_zlim(0, grid_size[2])
ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.set_zlabel("Z")
ax.set_title("3D Ray-Voxel Intersection with Ray Size {:.2f} and dty = {:.2f}".format(ray_size, dty))
ax.legend()
plt.show()

# Compute forward peaks for one dty position

In [ ]:
ds_max = 1.2
mask = None

rot_start = -89.975
rot_end = 90.9668
rot_step = 0.5
omega_angles = np.arange(rot_start, rot_end+rot_step/2, rot_step)
print("{} rotation angles in one dty position".format(omega_angles.shape[0]))

dty = 0.0
fwd_peaks_test = forward_projector.forward_peaks_voxels(beam, sample, omega_angles, ucell, pars, dty = dty, mask = None, ds_max = 1.2, plot_peaks=True, verbose = 1)
print(fwd_peaks_test.shape)

In [ ]:
io.write_fwd_peaks(fwd_peaks_test, output_folder = output_folder, fname_prefix='fpks_test')

In [ ]:
intensity_map = forward_projector.make_intensity_map(fwd_peaks_test[:, 5], fwd_peaks_test[:, 6], fwd_peaks_test[:, 23],
                                     x_range=[-0.18, 0.18], y_range=[-0.18, 0.18], pixel_size=1e-3)
print(intensity_map.shape)

In [ ]:
intensity_map = forward_projector.make_intensity_map(fwd_peaks_test[:, 18], fwd_peaks_test[:, 19], fwd_peaks_test[:, 23],
                                     x_range = [0, 2162], y_range = [0, 2068], pixel_size=1,)
print(intensity_map.shape)

In [ ]:
projs, projs_sum = forward_projector.make_projections_with_psf(
    fwd_peaks_test,
    omega_angles,
    image_size=(2162, 2068),
    detector='eiger',
    int_factors=(0.1065, 0.7807, 0.1065),
    sum_flag=True
)

In [ ]:
f, a = plt.subplots(1,2, sharex=True, sharey=True, figsize=(16, 8))

a[0].imshow(intensity_map, origin="lower", norm=LogNorm(vmin=10, vmax=1e4), interpolation="nearest")
a[0].set_title('(a) Sum of intensities without psf')

a[1].imshow(projs_sum, origin="lower", norm=LogNorm(vmin=10, vmax=1e4), interpolation="nearest")
a[1].set_title('(b) Sum of projections with psf')

a[1].set_xlim([200, 600])
a[1].set_ylim([200, 600])

In [ ]:
dty = 120.0
fwd_peaks_test = forward_projector.forward_peaks_voxels(beam, sample, omega_angles, ucell, pars, dty = dty, mask = None, ds_max = 1.2, plot_peaks=True, verbose = 1)
print(fwd_peaks_test.shape)

In [ ]:
intensity_map = forward_projector.make_intensity_map(fwd_peaks_test[:, 5], fwd_peaks_test[:, 6], fwd_peaks_test[:, 23],
                                     x_range=[-0.18, 0.18], y_range=[-0.18, 0.18], pixel_size=1e-3)
print(intensity_map.shape)

In [ ]:
io.write_fwd_peaks(fwd_peaks_test, output_folder = output_folder, fname_prefix='fpks_test')

In [ ]:
fwd_peaks = io.read_fwd_peaks(os.path.join(output_folder, 'fpks_test_dty_120p0.h5'))

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(15, 9))

sc = ax[0].scatter(fwd_peaks[:, 18], fwd_peaks[:, 19], c=fwd_peaks[:, 23], cmap='viridis', s=8)
ax[0].set_aspect('equal', 'box')
cb = f.colorbar(sc, ax=ax[0])
# cb.set_label('Intensity', fontsize = 20)
cb.ax.tick_params(labelsize=14)
ax[0].set_xlabel('fc', fontsize = 20)
ax[0].set_ylabel('sc', fontsize = 20)
ax[0].set_title('(a) Forward peaks on detector', fontsize = 20)
ax[0].tick_params(width=1.5, length=6, labelsize=14)
ax[0].invert_yaxis()

sc = ax[1].scatter(fwd_peaks[:, 5], fwd_peaks[:, 6], c=fwd_peaks[:, 23], cmap='viridis', s=8)
ax[1].set_aspect('equal', 'box')
cb = f.colorbar(sc, ax=ax[1])
cb.set_label('Intensity', fontsize = 20)
cb.ax.tick_params(labelsize=14)
ax[1].set_xlabel('X (mm)', fontsize = 20)
ax[1].set_ylabel('Y (mm)', fontsize = 20)
ax[1].set_title('(b) Forward peaks from the sample', fontsize = 20)
ax[1].tick_params(width=1.5, length=6, labelsize=14) 

f.tight_layout()
plt.show()

# Do testings with different omega steps

In [ ]:
ds_max = 1.2
mask = None

rot_start = -89.975
rot_end = 90.9668
rot_step = 0.25
omega_angles = np.arange(rot_start, rot_end+rot_step/2, rot_step)
print("{} rotation angles in one dty position".format(omega_angles.shape[0]))

dty = 0.0
fwd_peaks = forward_projector.forward_peaks_voxels(beam, sample, omega_angles, ucell, pars, dty = dty, mask = None, ds_max = 1.2, plot_peaks=True, verbose = 1)
print(fwd_peaks.shape)

In [ ]:
intensity_map = forward_projector.make_intensity_map(fwd_peaks[:, 5], fwd_peaks[:, 6], fwd_peaks[:, 23],
                                     x_range=[-0.18, 0.18], y_range=[-0.18, 0.18], pixel_size=1e-3)
print(intensity_map.shape)

In [ ]:
ds_max = 1.2
mask = None

rot_start = -89.975
rot_end = 90.9668
rot_step = 0.05
omega_angles = np.arange(rot_start, rot_end+rot_step/2, rot_step)
print("{} rotation angles in one dty position".format(omega_angles.shape[0]))

dty = 0.0
fwd_peaks = forward_projector.forward_peaks_voxels(beam, sample, omega_angles, ucell, pars, dty = dty, mask = None, ds_max = 1.2, plot_peaks=True, verbose = 1)
print(fwd_peaks.shape)

In [ ]:
io.write_fwd_peaks(fwd_peaks, output_folder = output_folder, fname_prefix='fpks')

# Testing a bigger beam size

In [ ]:
beam.FWHM = [0.002, 0.002]

In [ ]:
ds_max = 1.2
mask = None

rot_start = -89.975
rot_end = 90.9668
rot_step = 0.5
omega_angles = np.arange(rot_start, rot_end+rot_step/2, rot_step)
print("{} rotation angles in one dty position".format(omega_angles.shape[0]))

dty = 0.0
fwd_peaks_bigger_beam = forward_projector.forward_peaks_voxels(beam, sample, omega_angles, ucell, pars, dty = dty, mask = None, ds_max = 1.2, plot_peaks=True, verbose = 1)
print(fwd_peaks_bigger_beam.shape)

In [ ]:
fwd_peaks_bigger_beam.shape

In [ ]:
io.write_fwd_peaks(fwd_peaks_bigger_beam, output_folder = output_folder, fname_prefix='fpks_2um_beam')

In [ ]:
beam.FWHM = [0.003, 0.003]

In [ ]:
ds_max = 1.2
mask = None

rot_start = -89.975
rot_end = 90.9668
rot_step = 0.5
omega_angles = np.arange(rot_start, rot_end+rot_step/2, rot_step)
print("{} rotation angles in one dty position".format(omega_angles.shape[0]))

dty = 0.0
fwd_peaks_bigger_beam = forward_projector.forward_peaks_voxels(beam, sample, omega_angles, ucell, pars, dty = dty, mask = None, ds_max = 1.2, plot_peaks=True, verbose = 1)
print(fwd_peaks_bigger_beam.shape)

In [ ]:
beam.FWHM = [0.05, 0.05]

In [ ]:
ds_max = 1.2
mask = None

rot_start = -89.975
rot_end = 90.9668
rot_step = 0.5
omega_angles = np.arange(rot_start, rot_end+rot_step/2, rot_step)
print("{} rotation angles in one dty position".format(omega_angles.shape[0]))

dty = 0.0
fwd_peaks_bigger_beam = forward_projector.forward_peaks_voxels(beam, sample, omega_angles, ucell, pars, dty = dty, mask = None, ds_max = 1.2, plot_peaks=True, verbose = 1)
print(fwd_peaks_bigger_beam.shape)

In [ ]:
fwd_peaks_bigger_beam.shape

In [ ]:
io.write_fwd_peaks(fwd_peaks_bigger_beam, output_folder = output_folder, fname_prefix='fpks_50um_beam')

In [ ]:
beam.FWHM = [2, 2]

In [ ]:
ds_max = 1.2
mask = None

rot_start = -89.975
rot_end = 90.9668
rot_step = 0.5
omega_angles = np.arange(rot_start, rot_end+rot_step/2, rot_step)
print("{} rotation angles in one dty position".format(omega_angles.shape[0]))

dty = 0.0
fwd_peaks_bigger_beam = forward_projector.forward_peaks_voxels(beam, sample, omega_angles, ucell, pars, dty = dty, mask = None, ds_max = 1.2, plot_peaks=True, verbose = 1)
print(fwd_peaks_bigger_beam.shape)

In [ ]:
io.write_fwd_peaks(fwd_peaks_bigger_beam, output_folder = output_folder, fname_prefix='fpks_2000um_beam')

# Create projections

In [ ]:
rot_angle = -11.5282
t0 = time.time()
im1 = forward_projector.make_one_projection_with_psf(fwd_peaks, rot_angle, rot_step = 0.05, image_size=(2162, 2068), detector = 'frelon', int_factors = (0.1065, 0.7807, 0.1065))
print('It takes {}'.format(time.time() - t0))

In [ ]:
t0 = time.time()
im2 = forward_projector.make_one_projection_with_psf(fwd_peaks, rot_angle, rot_step = 0.05, image_size=(2162, 2068), detector = 'eiger', int_factors = (0.1065, 0.7807, 0.1065))
print('It takes {}'.format(time.time() - t0))

In [ ]:
f, a = plt.subplots(1, 2, figsize=(12, 6), sharex=True, sharey=True)
a[0].imshow(im1>0)
a[0].set_title('frelon')
a[1].imshow(im2>0)
a[1].set_title('eiger')
plt.show()

In [ ]:
t0 = time.time()
projs, _ = forward_projector.make_projections_with_psf(fwd_peaks, omega_angles, image_size=(2162, 2068),
                                                       detector = 'eiger', int_factors=(0.1065, 0.7807, 0.1065),
                                                      sum_flag=False)
print('It takes {}'.format(time.time() - t0))

# Make a forward_projector class and run calculations

In [ ]:
phase_name = 'Al'
opts = {
        "energy": 43.56,                 # [keV]
        "beam_size": [1e-3, 1e-3],       # [mm]
        "beam_profile": "gaussian",      # [-]
        "flux": 5e14,                    # [photons/s]
        "Lss": 0.0,                      # [mm]
        "min_misori": 3.0,               # [deg]
        "crystal_system": 'cubic',
        "remove_small_grains": True,
        "min_vol": 3,                    # [voxel]
        "rou": 2.7,                      # [g/cm^3]
        "mass_abs": 0.56685,             # [cm^2/g]
        "y0_offset": 0.0,                # [um]
        "exp_time": 0.002,               # [s]
        "rot_start": -89.975,            # [deg]
        "rot_end": 90.9668,              # [deg]
        "rot_step": 0.05,                # [deg]
        "sparse_omega": True,
        "halfy": 182.0,                  # [um]
        "dty_step": 1.0,                 # [um]
        "ds_max": 1.2,                   # [1/angstrom]
        "plot_peaks": False,
        "plot_flag": False,
        "detector": "eiger",
        "int_factors": (0.1065, 0.7807, 0.1065),
        "slurm_folder": "slurm_fwd_proj_Al"}

In [ ]:
fp = forward_projector.forward_projector(sample_filename, pars_filename, phase_name, output_folder=output_folder, detector_mask = None, to_sparse = False, **opts)

In [ ]:
t0 = time.time()
fp.run_single_dty(dty = 0.0)
t1 = time.time()
print('It takes {}'.format(t1 - t0))

In [ ]:
fp.read_fwd_peaks_from_file = False

In [ ]:
# allows overwrite by setting fp.read_fwd_peaks_from_file = False
t0 = time.time()
fp.run_single_dty(dty = 0.0)
t1 = time.time()
print('It takes {}'.format(t1 - t0))

In [ ]:
# run again, should be faster wit numba compiled
t0 = time.time()
fp.run_single_dty(dty = -120.0)
t1 = time.time()
print('It takes {}'.format(t1 - t0))

In [ ]:
dty_select = -120
if dty_select == 0:
    fwd_peaks = io.read_fwd_peaks(f'{output_folder}/fpks_dty_0p0.h5')
    fwd_peaks_3d = io.read_fwd_peaks(f'{output_folder}/fpks_3d_dty_0p0.h5')
else:
    fwd_peaks = io.read_fwd_peaks(f'{output_folder}/fpks_dty_-120p0.h5')
    fwd_peaks_3d = io.read_fwd_peaks(f'{output_folder}/fpks_3d_dty_-120p0.h5')
print(fwd_peaks.shape)
print(fwd_peaks_3d.shape)

In [ ]:
forward_projector.plot_fwd_peaks(fwd_peaks)

In [ ]:
intensity_map = forward_projector.make_intensity_map(fwd_peaks[:, 5], fwd_peaks[:, 6], fwd_peaks[:, 23],
                                     x_range=[-0.18, 0.18], y_range=[-0.18, 0.18], pixel_size=1e-3)
print(intensity_map.shape)

In [ ]:
intensity_map = forward_projector.make_intensity_map(fwd_peaks[:, 18], fwd_peaks[:, 19], fwd_peaks[:, 23],
                                     x_range = [0, 2162], y_range = [0, 2068], pixel_size=1,)
print(intensity_map.shape)

In [ ]:
omega_angles = np.arange(opts["rot_start"], opts["rot_end"] + opts["rot_step"]/2, np.max([opts["rot_step"], 0.5]))
print(omega_angles.shape)

In [ ]:
projs, projs_sum = forward_projector.make_projections_with_psf(
    fwd_peaks,
    omega_angles,
    image_size=(2162, 2068),
    detector='eiger',
    int_factors=(0.1065, 0.7807, 0.1065),
    sum_flag = True
)

In [ ]:
f, a = plt.subplots(1,2, sharex=True, sharey=True, figsize=(16, 8))

a[0].imshow(intensity_map, origin="lower", norm=LogNorm(vmin=10, vmax=1e4), interpolation="nearest")
a[0].set_title('(a) Sum of intensities without psf')

a[1].imshow(projs_sum, origin="lower", norm=LogNorm(vmin=10, vmax=1e4), interpolation="nearest")
a[1].set_title('(b) Sum of projections with psf')

a[1].set_xlim([200, 600])
a[1].set_ylim([200, 600])

# Compare with the experimental projections

In [ ]:
# experimental raw data
raw_h5 = '/data/visitor/ma6288/id11/20241119/RAW_DATA/A2050_DN_W340_nscope_5pct_strained/A2050_DN_W340_nscope_5pct_strained_full_slice/A2050_DN_W340_nscope_5pct_strained_full_slice.h5'

In [ ]:
# from -11.7282 to -11.4282 deg
# StartIndex=1565
# EndIndex=1571

# from -2.47853 to 2.5213 deg
StartIndex=1750
EndIndex=1850

In [ ]:
scan = f'{dty_select + 182 + 1}.1'
print(scan)

In [ ]:
# 183.1, dty = 0
# 63.1, dty = -120
projs_exp = io.read_images_from_h5(raw_h5, scan = scan, StartIndex=StartIndex, EndIndex=EndIndex)
print(projs_exp.shape)
projs_exp_sum = np.sum(projs_exp, axis = 0)

In [ ]:
omega_angles = np.arange(opts["rot_start"], opts["rot_end"] + opts["rot_step"]/2, opts["rot_step"])
print(omega_angles.shape)

In [ ]:
projs_simu, projs_simu_sum = forward_projector.make_projections_with_psf(
    fwd_peaks,
    omega_angles[StartIndex:EndIndex],
    image_size=(2162, 2068),
    detector='eiger',
    int_factors=(0.1065, 0.7807, 0.1065),
    sum_flag=True
)
print(projs_simu.shape)

In [ ]:
f, a = plt.subplots(1,2, sharex=True, sharey=True, figsize=(12, 6))

a[0].imshow(projs_exp_sum, origin="lower", norm=LogNorm(vmin=1, vmax=1000), interpolation="nearest")
a[0].set_title('(a) Exp')

a[1].imshow(projs_simu_sum, origin="lower", norm=LogNorm(vmin=1, vmax=1e4), interpolation="nearest")
a[1].set_title('(b) Simu')

if dty_select == 0:
    a[1].set_xlim([200, 350])
    a[1].set_ylim([1150, 1400])
else:
    a[1].set_xlim([200, 400])
    a[1].set_ylim([1300, 1500])
f.tight_layout()

# Convert fwd_peaks to sparse file

In [ ]:
opts_seg = forward_projector.get_opts_seg()

In [ ]:
opts_seg

In [ ]:
t0 = time.time()
projs, _ = forward_projector.make_projections_with_psf(
    fwd_peaks,
    omega_angles,
    image_size=(2162, 2068),
    detector='eiger',
    int_factors=(0.1065, 0.7807, 0.1065),
    sum_flag = False
)
t1 = time.time()
print('It takes {}'.format(t1 - t0))

In [ ]:
dty = fwd_peaks[0, 8]
destname = os.path.join(output_folder, 'fsparse_dty_' + str(round(dty, 2)).replace('.', 'p') + '.h5')
print(destname)

In [ ]:
print(fwd_peaks.shape, omega_angles.shape, projs.shape)

In [ ]:
t0 = time.time()
forward_projector.make_projs_and_sparse_file(fwd_peaks, destname, omega_angles, opts_seg, detector='eiger',
                                             image_size=(2162, 2068), int_factors=(0.1065, 0.7807, 0.1065))
t1 = time.time()
print('It takes {}'.format(t1 - t0))

In [ ]:
# filename for segmented sparse
destname = os.path.join(output_folder, 'fsparse_dty_' + str(round(dty, 2)).replace('.', 'p') + '_seg.h5')

In [ ]:
t0 = time.time()
forward_projector.segment_frms(projs, destname, detector='eiger', opts_seg = opts_seg)
t1 = time.time()
print('It takes {}'.format(t1 - t0))

# Assemble and label the peaks from the sparse file
# Note: this is only to illustrate the peak assembling, it is however recommended not to do so because of the speed

In [ ]:
forward_projector.assemble_sparsefiles(fp.output_folder, fp.dtys,
                                       outname_sparse=f'{output_folder}/fwd_sparse.h5',image_size=(2162, 2068))